In [2]:
! pip install ccxt

In [1]:
import json
import ccxt
import logging

import pandas as pd
import numpy as np

import time



In [5]:
config = json.load(open('../config.json'))

EXCHANGE = ccxt.binance({
    "apiKey": config["BINANCE_API_KEY"],
    "secret": config["BINANCE_SECRET_KEY"]
})



In [17]:
open_orders = EXCHANGE.fetch_open_orders("LRC/BUSD")

In [20]:
status = open_orders[0]["status"]

'open'

In [24]:
open_orders[0]

{'info': {'symbol': 'LRCBUSD',
  'orderId': '201663864',
  'orderListId': '-1',
  'clientOrderId': 'web_7f66b886f7c94cb0841cf35c5ad93feb',
  'price': '1.67770000',
  'origQty': '12.00000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': '1641477169031',
  'updateTime': '1641477169031',
  'isWorking': True,
  'origQuoteOrderQty': '0.00000000'},
 'id': '201663864',
 'clientOrderId': 'web_7f66b886f7c94cb0841cf35c5ad93feb',
 'timestamp': 1641477169031,
 'datetime': '2022-01-06T13:52:49.031Z',
 'lastTradeTimestamp': None,
 'symbol': 'LRC/BUSD',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 1.6777,
 'stopPrice': None,
 'amount': 12.0,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 12.0,
 'status': 'open',
 'fee': None,
 'trades': [],
 'fees': []}

In [21]:
def select_pairs(pairs, hours):
    """ Sort trading pairs by performance and return dataframe"""
    columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    
    pairs_perc = []

    for i, pair in enumerate(pairs):
        try:
            data = EXCHANGE.fetch_ohlcv(pair, "1h")
            df_temp = pd.DataFrame.from_records(data, columns=columns)
            df_temp.head(12)
            percentage = (df_temp["Close"].iloc[-1] / df_temp["Close"].iloc[-1-hours]) - 1
            pairs_perc.append([pair, percentage])
        except Exception as e:
            logging.warning(f"Could not fetch {pair}")
            logging.warning(f"{e}")
            

    df_pairs_perc = pd.DataFrame(pairs_perc, columns=["pair", "percentage"])
    df_pairs_perc = df_pairs_perc.sort_values(by=['percentage'], ascending=False)
    
    return df_pairs_perc

def filter_pairs(pairs):
    ''' Filter pairs and return only top 200 pairs'''
    # Add filtering for top XXX coins ?
    
    pairs_data = EXCHANGE.fetch_tickers()
    filtered_pairs = []
    
    for pair in pairs: 
        # De if statement kan worden uitgebreid met logica om te zien of het volume substantieel is.
        if pairs_data[pair]["baseVolume"]>0 or pairs_data[pair]["quoteVolume"]>0:
            filtered_pairs.append(pair)
    
    return filtered_pairs

def split_cash(cash, pairs):
    
    tickers = EXCHANGE.fetch_tickers()
    coin_allocation = {}
    isstable = []
    
    for pair in pairs:

        if pair not in isstable:
            ask_price = tickers[pair]["ask"]
            allocation = (1 / len(pairs)) # Allocation buckets maken zodat meerdere trading bots kunnen opereren op hetzelfde account
            available_cash = ( cash * 0.975 ) # Keep some cash in reserve to accomodate for price fluctuations
            coin_amount =  (allocation * available_cash) / ask_price 
            coin_allocation[pair] = [round(coin_amount, 8), ask_price]
        else:
            pass
    
    return coin_allocation

def log_warning(coin, value, bid_price):

    return None



In [36]:
def check_order_status(coin, amount, limit):
    # Kijken hoe we de relevante orders kunnen bijhouden
    # Gebruik maken van parse_dust_trade voor de dust trades
    
    price = amount * limit
    
    if price > 10:
        time.sleep(15)
        open_orders = EXCHANGE.fetch_open_orders(coin)
        if len(open_orders) > 0:
            status = open_orders[0]["status"]
            id = open_orders[0]["id"]

            if status == "open":
                print(f"Order {coin} of {amount} at {limit} not filled!")
                
                
                
                # EXCHANGE.cancel_order(id=id,symbol=coin)
                # EXCHANGE.create_limit_sell_order(coin, amount, limit*0.999)
                print(f"Order cancelled. New order placed at {limit*0.999}")
            
    
    return 
    

def sell_dust():
    return None

def check_market_down():
    return None

In [6]:
from binance.client import Client

binance_client = Client(api_key=config["BINANCE_API_KEY"], api_secret=config["BINANCE_SECRET_KEY"])


In [9]:
binance_client.transfer_dust(asset="FTM")

{'totalTransfered': '0.00593145',
 'totalServiceCharge': '0.00011862',
 'transferResult': [{'tranId': 94431351230,
   'fromAsset': 'FTM',
   'amount': '0.987',
   'transferedAmount': '0.00593145',
   'serviceChargeAmount': '0.00011862',
   'operateTime': 1642004441929}]}

In [ ]:
def sell_dust():
    
    binance_client = Client(api_key=config["BINANCE_API_KEY"], api_secret=config["BINANCE_SECRET_KEY"])
    balance = EXCHANGE.fetch_balance()
    
    not_keys = ['info', 'timestamp', 'datetime', 'free', 'used', 'total']
    tokens = [token for token in list(balance.keys()) if token not in not_keys]
    
    for token in tokens:
        try:
            if balance[token]["free"] > 0:
                binance_client.transfer_dust(asset=token)
        except Exception as e:
            print(f"Could not sell dust: {token}")
            print(e)

    if balance["BNB"]["free"] > 0:
        EXCHANGE.create_market_sell_order("BNB/BUSD", balance["BNB"]["free"])

    return None

In [10]:
balance = EXCHANGE.fetch_balance()

In [17]:
not_keys = ['info', 'timestamp', 'datetime', 'free', 'used', 'total']

tokens = [token for token in list(balance.keys()) if token not in not_keys]
tokens

['BTC',
 'LTC',
 'ETH',
 'NEO',
 'BNB',
 'QTUM',
 'EOS',
 'SNT',
 'BNT',
 'GAS',
 'BCC',
 'USDT',
 'HSR',
 'OAX',
 'DNT',
 'MCO',
 'ICN',
 'ZRX',
 'OMG',
 'WTC',
 'YOYOW',
 'LRC',
 'TRX',
 'SNGLS',
 'STRAT',
 'BQX',
 'FUN',
 'KNC',
 'CDT',
 'XVG',
 'IOTA',
 'SNM',
 'LINK',
 'CVC',
 'TNT',
 'REP',
 'MDA',
 'MTL',
 'SALT',
 'NULS',
 'SUB',
 'STX',
 'MTH',
 'ADX',
 'ETC',
 'ENG',
 'ZEC',
 'AST',
 'GNT',
 'DGD',
 'BAT',
 'DASH',
 'POWR',
 'BTG',
 'REQ',
 'XMR',
 'EVX',
 'VIB',
 'ENJ',
 'VEN',
 'ARK',
 'XRP',
 'MOD',
 'STORJ',
 'KMD',
 'RCN',
 'EDO',
 'DATA',
 'DLT',
 'MANA',
 'PPT',
 'RDN',
 'GXS',
 'AMB',
 'ARN',
 'BCPT',
 'CND',
 'GVT',
 'POE',
 'BTS',
 'FUEL',
 'XZC',
 'QSP',
 'LSK',
 'BCD',
 'TNB',
 'ADA',
 'LEND',
 'XLM',
 'CMT',
 'WAVES',
 'WABI',
 'GTO',
 'ICX',
 'OST',
 'ELF',
 'AION',
 'WINGS',
 'BRD',
 'NEBL',
 'NAV',
 'VIBE',
 'LUN',
 'TRIG',
 'APPC',
 'CHAT',
 'RLC',
 'INS',
 'PIVX',
 'IOST',
 'STEEM',
 'NANO',
 'AE',
 'VIA',
 'BLZ',
 'SYS',
 'RPX',
 'NCASH',
 'POA',
 'ONT',
 '

In [10]:
# EXCHANGE.load_markets()
EXCHANGE.currency("RVN")

{'id': 'RVN',
 'name': 'Ravencoin',
 'code': 'RVN',
 'precision': None,
 'info': {'coin': 'RVN',
  'depositAllEnable': True,
  'withdrawAllEnable': True,
  'name': 'Ravencoin',
  'free': '0.04554005',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoing': '0',
  'ipoable': '0',
  'storage': '0',
  'isLegalMoney': False,
  'trading': True,
  'networkList': [{'network': 'RVN',
    'coin': 'RVN',
    'withdrawIntegerMultiple': '0.00000001',
    'isDefault': True,
    'depositEnable': True,
    'withdrawEnable': True,
    'depositDesc': '',
    'withdrawDesc': '',
    'specialTips': '',
    'name': 'Ravencoin',
    'resetAddressStatus': False,
    'addressRegex': '^[Rr]{1}[A-Za-z0-9]{33,34}$',
    'addressRule': '',
    'memoRegex': '',
    'withdrawFee': '1',
    'withdrawMin': '2',
    'withdrawMax': '10000000000',
    'minConfirm': '30',
    'unLockConfirm': '200',
    'sameAddress': False}]},
 'active': True,
 'networks': [{'network': 'RVN',
   'coin': 'RVN',
   'withdrawI

In [6]:
# dust_list = str("KP3R/BUSD")

# EXCHANGE.load_markets()
# EXCHANGE.parse_dust_trade(dust_list)
balance = EXCHANGE.fetch_balance()

In [8]:
balance["SPELL"]

{'free': 0.258, 'used': 0.0, 'total': 0.258}

In [11]:
EXCHANGE.create_market_sell_order("SPELL/BNB", 0.258)

BadSymbol: binance does not have market symbol BNB/SPELL

In [25]:
from yahoo_fin.stock_info import get_top_crypto

# df_change = get_top_crypto()["% Change"]

def check_market_down():

    df_change = get_top_crypto().iloc[:20]["% Change"]

    if df_change.median() < 0:
        status = True
    else: 
        status = False

    return status

In [26]:
check_market_down()

True

In [24]:
pd.set_option('display.max_rows', None)

exempt_list = ["USDT-USD", "BUSD-USD"]

test = get_top_crypto()
test

,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply
0,BTC-USD,Bitcoin USD,40901.790000,-893.840000,-2.14,7.740040e+11,2.961300e+10,2.961300e+10,2.961300e+10,18923000.0
1,ETH-USD,Ethereum USD,3037.100000,-144.040000,-4.53,3.616510e+11,1.469500e+10,1.469500e+10,1.469500e+10,119078000.0
2,USDT-USD,Tether USD,1.000300,0.000100,0.01,7.831400e+10,5.732200e+10,5.732200e+10,5.732200e+10,78293000000.0
3,BNB-USD,Binance Coin USD,421.460000,-28.980000,-6.43,7.030000e+10,3.761000e+09,3.761000e+09,3.761000e+09,166801000.0
4,USDC-USD,USD Coin USD,1.000000,0.000300,0.03,4.344400e+10,3.968000e+09,3.968000e+09,3.968000e+09,43443000000.0
5,HEX-USD,HEX USD,0.245632,0.023612,10.64,4.259500e+10,5.231400e+07,5.231400e+07,5.231400e+07,173411000000.0
6,SOL-USD,Solana USD,134.470000,-4.920000,-3.53,4.186700e+10,2.231000e+09,2.231000e+09,2.231000e+09,311343000.0
7,ADA-USD,Cardano USD,1.143100,-0.081400,-6.65,3.829300e+10,1.241000e+09,1.241000e+09,1.241000e+09,33499000000.000004
8,XRP-USD,XRP USD,0.730272,-0.034704,-4.54,3.474400e+10,1.659000e+09,1.659000e+09,1.659000e+09,47577000000.0
9,DOT-USD,Polkadot USD,23.490000,-1.970000,-7.75,2.320300e+10,1.460000e+09,1.460000e+09,1.460000e+09,987579000.0


In [ ]:
def sell_dust():
    

In [38]:
def check_market_down():
    """ Check if the median of the 24hr return of the top 20 coins is down"""

    df_change = get_top_crypto().iloc[:20]["% Change"]

    if df_change.median() < 0:
        status = True
    else: 
        status = False

    return status

def sell_dust():
    """ Sell all dust for BNB and sell all BNB for BUSD"""
    
    binance_client = Client(api_key=config["BINANCE_API_KEY"], api_secret=config["BINANCE_SECRET_KEY"])
    balance = EXCHANGE.fetch_balance()
    
    not_keys = ['info', 'timestamp', 'datetime', 'free', 'used', 'total']
    tokens = [token for token in list(balance.keys()) if token not in not_keys]
    
    for token in tokens:
        try:
            if balance[token]["free"] > 0:
                binance_client.transfer_dust(asset=token)
        except Exception as e:
            print(f"Could not sell dust: {token}")
            print(e)

    if balance["BNB"]["free"] > 0:
        EXCHANGE.create_market_sell_order("BNB/BUSD", balance["BNB"]["free"])

    return None

def place_market_order(pairs):
    """ Place buy and sell ordered based on trend"""
    
    unfiltered_pairs = pairs["pair"].values
    market_down = check_market_down() 
    
    filtered_pairs = filter_pairs(unfiltered_pairs)
    top_10 = filtered_pairs[:10]
    balance = EXCHANGE.fetch_balance()
    df_balance = pd.DataFrame(balance["free"].items(), columns=["token", "balance"]) # check toevoegen om te zien of het meer dan $10 is 
    portfolio = df_balance[df_balance["balance"]>0]
        
    tickers = EXCHANGE.fetch_tickers()
    
    # Sell
    for token, value in dict(portfolio.values).items():
        if token != "BUSD": 
            coin = f"{token}/BUSD"
            bid_price = tickers[coin]["bid"]
            try:
                print(f"Sell: {coin}, Amount: {value}, Bid: {bid_price}")
                # EXCHANGE.create_limit_sell_order(coin, value, bid_price)
                EXCHANGE.create_market_sell_order(coin, value)
                check_order_status(coin, value, bid_price)
            except Exception as e:
                logging.warning(f"Could not sell Coin: {coin}, Amount: {value}, Bid:{bid_price}")
                logging.warning(e)
                
    cash = EXCHANGE.fetch_balance()["BUSD"]["free"]
    sell_dust()
    coin_allocation = split_cash(cash, top_10)
    
    # Buy
    for coin, value in coin_allocation.items():
        if market_down == False: # Check if overall market is down. If True, we will not reenter the market until next period
            try:
                print(f"Buy: {coin}, Amount: {value[0]}, Ask: {value[1]}")
                # EXCHANGE.create_limit_buy_order(coin, value[0], value[1])
                EXCHANGE.create_market_buy_order(coin, value[0])
            except Exception as e:
                logging.warning(f"Could not buy Coin: {coin}, Amount: {value[0]}, Ask:{value[1]}")
                logging.warning(e)


    
    return None

In [39]:
hours = 8 
pairs = [key for key in EXCHANGE.fetch_tickers().keys() if "/BUSD" in key]

df_pairs_sorted = select_pairs(pairs, hours)
place_market_order(df_pairs_sorted)

Sell: BTC/BUSD, Amount: 3.956e-05, Bid: 42818.27
Sell: LTC/BUSD, Amount: 0.00034013, Bid: 134.8
Sell: LRC/BUSD, Amount: 1.998, Bid: 1.7159
Sell: IOTA/BUSD, Amount: 0.964, Bid: 1.1864
Sell: LINK/BUSD, Amount: 0.00582, Bid: 23.93
Sell: POWR/BUSD, Amount: 0.774, Bid: 0.5266
Sell: REQ/BUSD, Amount: 0.875, Bid: 0.3066
Sell: XMR/BUSD, Amount: 0.00060358, Bid: 197.4
Sell: ENJ/BUSD, Amount: 16.0839, Bid: 2.52
Sell: MANA/BUSD, Amount: 11.988, Bid: 3.2864
Sell: XLM/BUSD, Amount: 0.3472517, Bid: 0.2657
Sell: WAVES/BUSD, Amount: 0.00687, Bid: 13.97
Sell: ELF/BUSD, Amount: 0.0703, Bid: 0.4168
Sell: SYS/BUSD, Amount: 0.873, Bid: 0.9837
Sell: XEM/BUSD, Amount: 0.632, Bid: 0.1214
Sell: IQ/BUSD, Amount: 0.263, Bid: 0.01129
Sell: POLY/BUSD, Amount: 0.0223, Bid: 0.4776
Sell: RVN/BUSD, Amount: 0.04554005, Bid: 0.11735
Sell: FET/BUSD, Amount: 0.752, Bid: 0.5407
Sell: CELR/BUSD, Amount: 0.429, Bid: 0.07429
Sell: ATOM/BUSD, Amount: 0.00104, Bid: 37.26
Sell: ONE/BUSD, Amount: 0.0514, Bid: 0.28078
Sell: FTM/BU